In [239]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
import re

%matplotlib inline

In [240]:
input_folder_db = '/media/john/Data/Tensor_Invest_Fund/data/'

stocks = ['NQ', 'ES']
stocks = ['ES']
ticker = {'NQ': 'MNQ', 'ES':'MES'} 

folder_path = 'Micro_Mini_Future_'
parquet_file_name = '_Micro_Mini_Future_'

years_filter = [2021,2022]


dfs = []

for stock in stocks:
    for year in years_filter:

        file_name = os.path.join(input_folder_db, f'{folder_path}{stock}', f'{year}{parquet_file_name}{stock}.parquet'  )

        print("Reading file", file_name)
        
        df = pd.read_parquet(file_name)

        df['ticker'] = ticker[stock]

        dfs.append(df)

        del df

data = pd.concat(dfs, ignore_index = True)


Reading file /media/john/Data/Tensor_Invest_Fund/data/Micro_Mini_Future_ES/2021_Micro_Mini_Future_ES.parquet
Reading file /media/john/Data/Tensor_Invest_Fund/data/Micro_Mini_Future_ES/2022_Micro_Mini_Future_ES.parquet


In [241]:
def convert_time_format(x):

    if re.search('\d{6}', x):
        return x[:2] + ":" + x[2:4]
    else:
        return x

data.loc[:,'Time'] = data['Time'].apply(lambda x: convert_time_format(x) )

# Convert to Datetime
data['timestamp'] = pd.to_datetime((data['Date'].astype(str) + " " + data['Time'].astype(str) ), format = "%Y-%m-%d %H:%M")

# Convert Time to tuple
data['Time_tuple'] = data['Time'].apply(lambda x: tuple([int(x[:2]) , int(x[-2:])] ))

In [242]:
data[data['ticker'] == 'MES'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390964 entries, 0 to 390963
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Date        390964 non-null  datetime64[ns]
 1   Time        390964 non-null  object        
 2   Open        390964 non-null  float64       
 3   High        390964 non-null  float64       
 4   Low         390964 non-null  float64       
 5   Close       390964 non-null  float64       
 6   Vol         390964 non-null  float64       
 7   OI          0 non-null       float64       
 8   year        390964 non-null  int64         
 9   month       390964 non-null  int64         
 10  day         390964 non-null  int64         
 11  hour        390964 non-null  object        
 12  minute      390964 non-null  object        
 13  ticker      390964 non-null  object        
 14  timestamp   390964 non-null  datetime64[ns]
 15  Time_tuple  390964 non-null  object        
dtypes:

In [243]:
data['Date'].max()

Timestamp('2022-02-09 00:00:00')

# Model 1

Defining Profit-Stop Strategy 

Profit-Strop Ratio 3:1

In [5]:
# TODO: Add for short 

# Target points
tp = 10

# Stop points
sp = 10  # [0,10] - nidrieges Risiko / > 10 hohes Risiko

# Window Time Frame ( in minutes)
w = 5

strategy = f"t{tp}-r{sp}_w{w}"

# Select Ticker
ticker = 'MNQ'

# Select Training/Testing Data
start_date = datetime(2021,6,1)
end_date = datetime(2021,10,14)

time_filter = True

start_time = tuple([15,10])
end_time = tuple([21,50])

max_end_win_time = tuple([end_time[0],end_time[1]-w])

# Filter for ticke MNQ
df_input = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)].copy(deep = True)

if time_filter: 
    df_input = df_input[(df_input['Time_tuple'] >= start_time) & (df_input['Time_tuple'] <= end_time) ]

df_input = df_input[df_input['ticker'] == ticker].copy(deep = True).reset_index(drop = True)

In [6]:
max_end_win_time

(21, 45)

In [7]:
min_date = df_input['datetime'].min()
max_date = df_input['datetime'].max()

print(" Data Available from ", min_date, " to ", max_date)
print(" Time available from", df_input['Time_tuple'].min(), " until ", df_input['Time_tuple'].max())
print(" Number of possibble entries: ", df_input.shape[0])

 Data Available from  2021-06-01 15:10:00  to  2021-10-14 21:50:00
 Time available from (15, 10)  until  (21, 50)
 Number of possibble entries:  38958


In [8]:
df_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38958 entries, 0 to 38957
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        38958 non-null  datetime64[ns]
 1   Time        38958 non-null  object        
 2   Open        38958 non-null  float64       
 3   High        38958 non-null  float64       
 4   Low         38958 non-null  float64       
 5   Close       38958 non-null  float64       
 6   Vol         38958 non-null  float64       
 7   OI          38958 non-null  float64       
 8   year        38958 non-null  int64         
 9   month       38958 non-null  int64         
 10  day         38958 non-null  int64         
 11  hour        38958 non-null  object        
 12  minute      38958 non-null  object        
 13  ticker      38958 non-null  object        
 14  datetime    38958 non-null  datetime64[ns]
 15  Time_tuple  38958 non-null  object        
dtypes: datetime64[ns](2), 

In [9]:
def model_1_price_change_region(x, bar_price_type):

    if x[bar_price_type] >= x['target_price']:
        return 2

    if x[bar_price_type] < x['target_price'] and x[bar_price_type] >= x['entry_price']:
        return 1
    
    if x[bar_price_type] > x['stop_price'] and x[bar_price_type] < x['entry_price']:
        return 0

    if x[bar_price_type] <= x['stop_price']:
        return -1

def model_1_target_classification(df_window):

    Z_matrix = np.array(df_window[['price_region_high','price_region_low']])

    # The Target profit cannot be reached in the same minute as entry position
    condition_profit = 2 in Z_matrix[1:]

    if condition_profit:
        return 1
    else:
        return 0

def model_1_risk_classification(df_window):

    Z_matrix = np.array(df_window[['price_region_high','price_region_low']])

    # There should not be any -1
    condition_low_risk = 0 in Z_matrix
    condition_high_risk = -1 in Z_matrix

    # First test for high risk condition
    # if high risk condition is not met 
    # then it should be a low risk condition
    if condition_high_risk:
        return 0
    elif condition_low_risk:
        return 1
    else:
        return 1


def model1(df_input, max_end_win_time, time_filter, verbose = 0):
    """
    """

    # Define Entry at Market Price as mean values between High and Low
    df_input['entry_market'] = df_input[['High','Low']].mean(axis = 1)
    # Define Target Price
    df_input['target'] = df_input['entry_market'].apply(lambda x: x + tp)
    # Define Stop price
    df_input['stop'] =  df_input['entry_market'].apply(lambda x: x - sp)

    # Set Limit for Screener
    # Screener alwasy needs w bars in the future
    # to be able to classify a point
    limit_loop = df_input.shape[0] - w -1

    df_windows = []

    df_classified = pd.DataFrame()

    for index, row in df_input.iterrows():

        if index == limit_loop:
            break
            
        if time_filter:
            if row['Time_tuple'] > max_end_win_time:
                continue
        
        target = row['target']
        entry_market = row['entry_market'] 
        stop = row['stop'] 
        _datetime = row['datetime']

        df_window = df_input.iloc[index + 1:index + w +1,:].copy(deep = True)

        df_window.drop( columns = ['entry_market','target','stop'], inplace = True)

        df_window.loc[:,'target_price'] = target
        df_window.loc[:,'stop_price'] = stop
        df_window.loc[:,'entry_price'] = entry_market
        df_window.loc[:,'entry_time'] = _datetime

        # Classify Price Region for all highs
        df_window.loc[:,'price_region_high'] = (df_window[['High','target_price','stop_price','entry_price']]
                                .apply(lambda x: model_1_price_change_region(x, bar_price_type = 'High'), 
                                axis = 1 ))

        # Classify Price Region for all lows
        df_window.loc[:,'price_region_low'] = (df_window[['Low','target_price','stop_price','entry_price']]
                                        .apply(lambda x: model_1_price_change_region(x, bar_price_type = 'Low'), 
                                        axis = 1 ))

        entry_type = model_1_target_classification(df_window)
        risk_type  = model_1_risk_classification(df_window)
        
        df_class = pd.DataFrame({'datetime': [_datetime]})

        # Entry can be: 1: Target filled, 0: Target not filled
        df_class['entry_type'] = entry_type
        df_class['entry_market'] = entry_market

        # Risk can be 1: Low Risk, 0: High Risk
        df_class['risk_type'] = risk_type

        if df_classified.empty:
            df_classified = df_class.copy()
        else:
            df_classified = df_classified.append(df_class)

        if verbose > 0:
            print(index, ": " ,row['datetime'], " - Entry Price" , row['entry_market'], "is an entry type", entry_type)

        df_windows.append(df_window)

        #break

    # For all Z(t) Values
    # Concat them in one data frame
    df_windows = pd.concat(df_windows, ignore_index = True)

    # Join Entry Classification back to input data frame
    df_input = pd.merge( df_input, df_classified, on = ['datetime','entry_market'], how = 'left')

    return df_input, df_windows

        

In [10]:
df_input_class, df_windows = model1(df_input, max_end_win_time, time_filter, verbose = 0)

In [11]:
df_input_class['entry_type'].value_counts()

0.0    28017
1.0    10460
Name: entry_type, dtype: int64

In [12]:
df_input_class['risk_type'].value_counts()

1.0    27379
0.0    11098
Name: risk_type, dtype: int64

In [13]:
df_input_class.columns

Index(['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Vol', 'OI', 'year',
       'month', 'day', 'hour', 'minute', 'ticker', 'datetime', 'Time_tuple',
       'entry_market', 'target', 'stop', 'entry_type', 'risk_type'],
      dtype='object')

In [14]:
# t = '2021-09-20 21:10:00'
# df_input[df_input['datetime'] == t ]
# df_windows[df_windows['entry_time'] == t][['entry_time','High','Low','entry_price','target_price','stop_price','Time_tuple','price_region_high','price_region_low']]

## Data Storage

In [15]:
output_location_entry = os.path.join(input_folder_db, 'Model1',  ticker, 'Bar_Entry_Positions_{}_{}_{}.parquet'
                            .format(start_date.strftime('%Y-%m-%d'), 
                            end_date.strftime('%Y-%m-%d'),
                            strategy ))

output_location_windows = os.path.join(input_folder_db, 'Model1',  ticker, 'Windows_Price_Region_{}_{}_{}.parquet'
                            .format(start_date.strftime('%Y-%m-%d'), 
                            end_date.strftime('%Y-%m-%d'),
                            strategy ))

print(output_location_entry)

df_input_class['Time_tuple'] = df_input_class['Time_tuple'].astype(str)
df_input_class.to_parquet(output_location_entry , engine = 'fastparquet', compression = 'gzip')

df_windows['Time_tuple'] = df_windows['Time_tuple'].astype(str)
df_windows.to_parquet(output_location_windows , engine = 'fastparquet', compression = 'gzip')

../../data/Model1\MNQ\Bar_Entry_Positions_2021-06-01_2021-10-14_t10-r10_w5.parquet


In [16]:
print("data Stored at:")

print(" Classification entry data: ", output_location_entry)

print(" Windows classification:", output_location_windows)

data Stored at:
 Classification entry data:  ../../data/Model1\MNQ\Bar_Entry_Positions_2021-06-01_2021-10-14_t10-r10_w5.parquet
 Windows classification: ../../data/Model1\MNQ\Windows_Price_Region_2021-06-01_2021-10-14_t10-r10_w5.parquet


In [17]:
output_location_entry

'../../data/Model1\\MNQ\\Bar_Entry_Positions_2021-06-01_2021-10-14_t10-r10_w5.parquet'